In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import fisher_exact
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [ ]:
combined_flights_2022 = pd.read_parquet("Combined_Flights_2022.parquet")
combined_flights_2021 = pd.read_parquet("Combined_Flights_2021.parquet")
combined_flights_2020 = pd.read_parquet("Combined_Flights_2020.parquet")
combined_flights_2019 = pd.read_parquet("Combined_Flights_2019.parquet")
combined_flights_2018 = pd.read_parquet("Combined_Flights_2018.parquet")

In [ ]:
pattern_2018 = combined_flights_2018[["FlightDate","Airline", "Month", "DayOfWeek"]]
pattern_2019 = combined_flights_2019[["FlightDate","Airline", "Month", "DayOfWeek"]]
pattern_2020 = combined_flights_2020[["FlightDate","Airline", "Month", "DayOfWeek"]]
pattern_2021 = combined_flights_2021[["FlightDate","Airline", "Month", "DayOfWeek"]]
pattern_2022 = combined_flights_2022[["FlightDate","Airline", "Month", "DayOfWeek"]]

In [ ]:
combined_pattern = pd.concat([pattern_2018, pattern_2019, pattern_2020, pattern_2021, pattern_2022])

In [ ]:
counts_by_month = combined_pattern.groupby('Month').count()

In [ ]:
counts_by_month = counts_by_month.reset_index()

counts_by_month.rename(columns={'index': 'Month'}, inplace=True)


In [ ]:
plt.bar(counts_by_month["Month"], counts_by_month["FlightDate"])
plt.xticks(ticks=counts_by_month["Month"], labels=counts_by_month["Month"])
plt.xlabel("Month")
plt.ylabel("Counts")
plt.title("Number of flights per Month from 2018-2022")

In [ ]:
counts_by_week = combined_pattern.groupby('DayOfWeek').count()
counts_by_week = counts_by_week.reset_index()

In [ ]:
plt.bar(counts_by_week["DayOfWeek"], counts_by_week["FlightDate"])
plt.xlabel("Day of the Week")
plt.ylabel("Counts")
plt.title("Number of flights per Day of the Week from 2018-2022")

In [ ]:
combined_flights_2018['total delay'] = combined_flights_2018['DepDelayMinutes'] + combined_flights_2018['ArrDelayMinutes']
combined_flights_2019['total delay'] = combined_flights_2019['DepDelayMinutes']+ combined_flights_2019['ArrDelayMinutes']
combined_flights_2020['total delay'] = combined_flights_2020['DepDelayMinutes']+ combined_flights_2020['ArrDelayMinutes']
combined_flights_2021['total delay'] = combined_flights_2021['DepDelayMinutes']+ combined_flights_2021['ArrDelayMinutes']
combined_flights_2022['total delay'] = combined_flights_2022['DepDelayMinutes']+ combined_flights_2022['ArrDelayMinutes']

In [ ]:
delay_config_2018 = combined_flights_2018[["FlightDate","Airline","total delay"]]
delay_config_2019 = combined_flights_2019[["FlightDate","Airline","total delay"]]
delay_config_2020 = combined_flights_2020[["FlightDate","Airline","total delay"]]
delay_config_2021 = combined_flights_2021[["FlightDate","Airline","total delay"]]
delay_config_2022 = combined_flights_2022[["FlightDate","Airline","total delay"]]

In [ ]:
combined_delay = pd.concat([delay_config_2018, delay_config_2019, delay_config_2020, delay_config_2021, delay_config_2022])

In [ ]:
airline_counts = combined_delay.groupby('Airline').count()
airline_counts = airline_counts.reset_index()

In [ ]:
plt.bar(airline_counts["Airline"], airline_counts["FlightDate"])
plt.xlabel("Airline")
plt.xticks(rotation = 90)
plt.ylabel("Counts")
plt.title("Number of flights per Airline from 2018-2022")

In [ ]:
combined_delay["count"] = combined_delay["Airline"].map(combined_delay["Airline"].value_counts())
combined_delay["normalized_count"] = (combined_delay["count"] - combined_delay["count"].min()) / (combined_delay["count"].max() - combined_delay["count"].min())
combined_delay

In [ ]:
airline_delays = combined_delay.groupby("Airline").sum(numeric_only=True)[["total delay", "normalized_count"]].reset_index()

airline_delays["normalized_total_delay"] = (airline_delays["total delay"] - airline_delays["total delay"].mean()) / \
                                           airline_delays["total delay"].std()
plt.figure(figsize=(7, 5))
plt.bar(airline_delays["Airline"], airline_delays["normalized_total_delay"])


plt.xlabel("Airlines")
plt.ylabel("Normalized Total Delay")
plt.xticks(rotation =90)
plt.title("Normalized Total Delay Minutes by Airline")

plt.show()

In [ ]:
#Fisher's Exact Test 

#get flightdate, cancelled, diverted 
binary_2018 = combined_flights_2018[["FlightDate","Cancelled", "Diverted"]]
binary_2019 = combined_flights_2019[["FlightDate","Cancelled", "Diverted"]]
binary_2020 = combined_flights_2020[["FlightDate","Cancelled", "Diverted"]]
binary_2021 = combined_flights_2021[["FlightDate","Cancelled", "Diverted"]]
binary_2022 = combined_flights_2022[["FlightDate","Cancelled", "Diverted"]]
#concat 
binary_all = pd.concat([binary_2018, binary_2019, binary_2020, binary_2021, binary_2022])
#make contingency table 
contingency_table = pd.crosstab(binary_all['Cancelled'], binary_all['Diverted'])
#set hypothesis
#H0 = cancelled and diverted are not independent 
#Ha cancelled and diverted are indepeneent 
#perform test
odds_ratio, p_value = fisher_exact(contingency_table, alternative='two-sided')

print(f"Odds Ratio: {odds_ratio}")
print(f"P-value: {p_value}")


In [ ]:
contingency_table

In [ ]:
#since pvalue is less than 0.05, there is evidence to conclude that there is significant association between cancelled and diverted 

In [ ]:
# add flight delays bar plots
flightdelay_2018 = combined_flights_2018[(combined_flights_2018["DepDelay"] > 0) | (combined_flights_2018["ArrDelay"] > 0)
][["FlightDate", "Airline", "Month", "DayOfWeek","DepDelay", "ArrDelay"]]
flightdelay_2019 = combined_flights_2019[(combined_flights_2019["DepDelay"] > 0) | (combined_flights_2019["ArrDelay"] > 0)
][["FlightDate", "Airline","Month", "DayOfWeek", "DepDelay", "ArrDelay"]]
flightdelay_2020 = combined_flights_2020[(combined_flights_2020["DepDelay"] > 0) | (combined_flights_2020["ArrDelay"] > 0)
][["FlightDate", "Airline","Month", "DayOfWeek", "DepDelay", "ArrDelay"]]
flightdelay_2021 = combined_flights_2021[(combined_flights_2021["DepDelay"] > 0) | (combined_flights_2021["ArrDelay"] > 0)
][["FlightDate", "Airline","Month", "DayOfWeek", "DepDelay", "ArrDelay"]]
flightdelay_2022 = combined_flights_2022[(combined_flights_2022["DepDelay"] > 0) | (combined_flights_2022["ArrDelay"] > 0)
][["FlightDate", "Airline","Month", "DayOfWeek", "DepDelay", "ArrDelay"]]
 

In [ ]:
combined_delays = pd.concat([flightdelay_2018, flightdelay_2019, flightdelay_2020, flightdelay_2021, flightdelay_2022])

In [ ]:
delays_by_month = combined_delays.groupby('Month').count()
delays_by_month = delays_by_month.reset_index()

In [ ]:
delays_by_week = combined_delays.groupby('DayOfWeek').count()
delays_by_week = delays_by_week.reset_index()

In [ ]:
plt.bar(delays_by_week["DayOfWeek"], delays_by_week["FlightDate"])
plt.xticks(ticks=delays_by_week["DayOfWeek"], labels=delays_by_week["DayOfWeek"])
plt.xlabel("Day of Week")
plt.ylabel("Counts")
plt.title("Number of flight delays per Day of Week from 2018-2022")